In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


microsoft_data=pd.read_csv('Microsoft_Data.csv')




In [12]:
# chi2 square test between OsBuild and HasDetections

from scipy.stats import chi2_contingency

contingency_table= pd.crosstab(microsoft_data['OsBuild'],microsoft_data['HasDetections'])

results=chi2_contingency(contingency_table)


print('p-value:',results[1])




p-value: 1.434114178500418e-17


In [3]:
cols_to_drop=[]

cols_with_na_grt_50=list(missing_value_percentage_df[missing_value_percentage_df['Missing_value_percentage']>50].index)

for col in microsoft_data.columns:
    if microsoft_data[col].nunique()==microsoft_data.shape[0] or microsoft_data[col].nunique()==1:
        cols_to_drop.append(col)
        

columns_to_drop=cols_to_drop+cols_with_na_grt_50

microsoft_data.drop(columns=columns_to_drop,inplace=True)


In [4]:
for col in microsoft_data.columns:
    if col!='Census_InternalPrimaryDiagonalDisplaySizeInInches':
        microsoft_data[col]=microsoft_data[col].astype(object)
        
microsoft_data['Census_InternalPrimaryDiagonalDisplaySizeInInches'].dtype


dtype('float64')

In [5]:
# Demonstration of  KFoldCV 

from sklearn.model_selection import KFold

kf=KFold(n_splits=5,shuffle=True)

for train_index,val_index in kf.split(microsoft_data,y):
    print(train_index,val_index)

[    0     1     2 ... 56753 56754 56755] [    8    11    21 ... 56740 56747 56748]
[    0     1     2 ... 56752 56753 56755] [    4     7     9 ... 56743 56749 56754]
[    0     2     4 ... 56753 56754 56755] [    1     3    20 ... 56741 56744 56746]
[    1     3     4 ... 56753 56754 56755] [    0     2    10 ... 56734 56750 56751]
[    0     1     2 ... 56750 56751 56754] [    5     6    12 ... 56752 56753 56755]


In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(microsoft_data,y,test_size=0.2,random_state=43)

# Filling missing values

for col in X_train.columns:
    if X_train[col].dtype=='object':
        X_train[col]=X_train[col].fillna(X_train[col].mode()[0]).astype(object)
        X_test[col]=X_test[col].fillna(X_train[col].mode()[0]).astype(object)
        
    else:
        X_train[col]=X_train[col].fillna(X_train[col].mean())
        X_test[col]=X_test[col].fillna(X_train[col].mean())

  

In [7]:
cat_cols= [col for col in X_train.columns if X_train[col].dtype=='object']

one_hot_train=pd.get_dummies(X_train[cat_cols]) # one-hot encoding

#one_hot_train

In [8]:
one_hot_train.shape
#one_hot_train

(45404, 63006)

In [9]:
one_hot_test=pd.get_dummies(X_test[cat_cols])

In [10]:
one_hot_test.shape
#one_hot_test

(11352, 25674)

In [11]:
X_train_final,X_test_final=one_hot_train.align(one_hot_test,join='inner',axis=1,fill_value=0)



In [12]:
X_train_final['Census_InternalPrimaryDiagonalDisplaySizeInInches']=X_train['Census_InternalPrimaryDiagonalDisplaySizeInInches']
X_test_final['Census_InternalPrimaryDiagonalDisplaySizeInInches']=X_test['Census_InternalPrimaryDiagonalDisplaySizeInInches']

In [ ]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier

gbc=GradientBoostingClassifier()





dtc=DecisionTreeClassifier()

hyp_values={'max_depth':[5,7,9,10,11],
           'min_samples_split':[4,5,6,7],
           'min_samples_leaf':[3,4,5,6],
           'max_leaf_nodes':[7,8,9,10]
           }

gcv=RandomizedSearchCV(dtc,param_distributions=hyp_values,cv=5,verbose=42,n_iter=100)

gcv.fit(X_train_final,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] min_samples_split=4, min_samples_leaf=6, max_leaf_nodes=10, max_depth=5 
[CV]  min_samples_split=4, min_samples_leaf=6, max_leaf_nodes=10, max_depth=5, score=1.0, total=  20.0s
[CV] min_samples_split=4, min_samples_leaf=6, max_leaf_nodes=10, max_depth=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.0s remaining:    0.0s


[CV]  min_samples_split=4, min_samples_leaf=6, max_leaf_nodes=10, max_depth=5, score=1.0, total=  15.9s
[CV] min_samples_split=4, min_samples_leaf=6, max_leaf_nodes=10, max_depth=5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   50.8s remaining:    0.0s


[CV]  min_samples_split=4, min_samples_leaf=6, max_leaf_nodes=10, max_depth=5, score=1.0, total=  15.0s
[CV] min_samples_split=4, min_samples_leaf=6, max_leaf_nodes=10, max_depth=5 


...

[CV]  min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=9, max_depth=9, score=1.0, total=  20.2s
[CV] min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=9, max_depth=9 
[CV]  min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=9, max_depth=9, score=1.0, total=  20.1s
[CV] min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5, score=1.0, total=  20.2s
[CV] min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5, score=1.0, total=  20.0s
[CV] min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5, score=1.0, total=  20.4s
[CV] min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=9, max_depth=5, score=1.0, total=  20.

[CV]  min_samples_split=7, min_samples_leaf=3, max_leaf_nodes=7, max_depth=9, score=1.0, total=  19.6s
[CV] min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5, score=1.0, total=  20.3s
[CV] min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5, score=1.0, total=  19.8s
[CV] min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5, score=1.0, total=  19.9s
[CV] min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5, score=1.0, total=  20.0s
[CV] min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5 
[CV]  min_samples_split=5, min_samples_leaf=5, max_leaf_nodes=9, max_depth=5, score=1.0, total=  19.

[CV]  min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10, score=1.0, total=  18.7s
[CV] min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10 
[CV]  min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10, score=1.0, total=  18.3s
[CV] min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10 
[CV]  min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10, score=1.0, total=  18.0s
[CV] min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10 
[CV]  min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10, score=1.0, total=  17.4s
[CV] min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10 
[CV]  min_samples_split=6, min_samples_leaf=3, max_leaf_nodes=7, max_depth=10, score=1.0, total=  18.6s
[CV] min_samples_split=7, min_samples_leaf=4, max_leaf_nodes=8, max_depth=7 
[CV]  min_samples_split=7, min_samples_leaf=4, max_leaf_nodes=8, max_depth=7, score=1.0, to

[CV]  min_samples_split=7, min_samples_leaf=6, max_leaf_nodes=7, max_depth=9, score=1.0, total=  19.9s
[CV] min_samples_split=7, min_samples_leaf=6, max_leaf_nodes=7, max_depth=9 
[CV]  min_samples_split=7, min_samples_leaf=6, max_leaf_nodes=7, max_depth=9, score=1.0, total=  19.6s
[CV] min_samples_split=7, min_samples_leaf=6, max_leaf_nodes=7, max_depth=9 
[CV]  min_samples_split=7, min_samples_leaf=6, max_leaf_nodes=7, max_depth=9, score=1.0, total=  19.9s
[CV] min_samples_split=7, min_samples_leaf=6, max_leaf_nodes=7, max_depth=9 
[CV]  min_samples_split=7, min_samples_leaf=6, max_leaf_nodes=7, max_depth=9, score=1.0, total=  19.7s
[CV] min_samples_split=4, min_samples_leaf=4, max_leaf_nodes=9, max_depth=9 
[CV]  min_samples_split=4, min_samples_leaf=4, max_leaf_nodes=9, max_depth=9, score=1.0, total=  20.2s
[CV] min_samples_split=4, min_samples_leaf=4, max_leaf_nodes=9, max_depth=9 
[CV]  min_samples_split=4, min_samples_leaf=4, max_leaf_nodes=9, max_depth=9, score=1.0, total=  20.

[CV]  min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=8, max_depth=5, score=1.0, total=  20.0s
[CV] min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=8, max_depth=5 
[CV]  min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=8, max_depth=5, score=1.0, total=  20.0s
[CV] min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=8, max_depth=5 
[CV]  min_samples_split=6, min_samples_leaf=6, max_leaf_nodes=8, max_depth=5, score=1.0, total=  18.9s
[CV] min_samples_split=5, min_samples_leaf=4, max_leaf_nodes=7, max_depth=10 
[CV]  min_samples_split=5, min_samples_leaf=4, max_leaf_nodes=7, max_depth=10, score=1.0, total=  20.0s
[CV] min_samples_split=5, min_samples_leaf=4, max_leaf_nodes=7, max_depth=10 
[CV]  min_samples_split=5, min_samples_leaf=4, max_leaf_nodes=7, max_depth=10, score=1.0, total=  20.1s
[CV] min_samples_split=5, min_samples_leaf=4, max_leaf_nodes=7, max_depth=10 


In [ ]:
import dataricks.koalas as ks

ks.read_csv('')

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression()
logreg.fit(X_train_final,y_train)

In [ ]:
logreg_pred=logreg.predict(X_test_final)

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score
confusion_matrix(y_test,logreg_pred)

precision_score(y_test,logreg_pred)
recall_score(y_test,logreg_pred)
f1_score(y_test,logreg_pred)

# Examples to demonstrate one hot encoding

In [ ]:


train=['Hyd','Chennai','Hyd','Bangalore','Bangalore','Pune','Chennai','Hyd','Pune','Amsterdam']

test=['Hyd','Chennai','Hyd','Ahmedabad','Pune','Chennai','Hyd','Bangalore','Berlin']




In [ ]:
pd.get_dummies(train)


In [ ]:
pd.get_dummies(test)